In [24]:
import simpy
import random
import matplotlib.pyplot as plt

MAXSIMTIME = 100 # max idle time


class Job:
    def __init__(self, id, name, arrtime, duration):
        self.name = name
        self.tid = id
        self.arrtime = arrtime
        self.duration = duration
        self.responseTime = -1
        
    def __str__(self):
        return '%s at %d, length %d' %(self.name, self.arrtime, self.duration)

class Server:
    def __init__(self, env, strat = 'FIFO'):
        self.env = env
        self.strat = strat
        self.Jobs = list(())
        self.serversleeping = None

        ''' statistics '''
        self.waitingTime = 0
        self.idleTime = 0
        self.jobsDone = 0

        ''' register a new server process '''
        self.servproc = env.process(self.serve(env))

        
    def serve(self, env):
        while True:
            ''' do nothing, just change server to idle
              and then yield a wait event which takes infinite time
            '''
            if len( self.Jobs ) == 0 :
                self.serversleeping = env.process( self.waiting( self.env ))
                t1 = self.env.now
                yield self.serversleeping
                ''' accumulate the server idle time'''
                self.idleTime += self.env.now - t1
            else:
                ''' get the first job to be served'''
                # FIFO 
                j = self.Jobs.pop(0)

                ''' sum up the waiting time'''
                self.waitingTime += self.env.now - j.arrtime
                #######

                ''' yield an event for the job finish'''
                yield self.env.timeout( j.duration )

                ''' sum up the jobs done '''
                self.jobsDone += 1


                """ as the process is finished by CPU, it returns back to Terminal
                    therefore, server must wake up that Terminal which is sleeping """
                self.tlist[j.tid].waitrtproc.interrupt()

                if self.NJobDone == self.NJob:
                    """ reach threshold """
                    self.endTime = env.now
                    print("End at %.8f" % self.endTime)

                    acc = 0
                    for i in range(self.NTerminal):
                        self.tlist[i].done()
                        acc += self.tlist[i].NJobGen

                    self.avgResTime /= acc
                    return
    def waiting(self, env):
        try:
            print("Server is idle at %5.5f" % env.now)
            yield env.timeout(MAXSIMTIME)
        except simpy.Interrupt:
            print("A new job comes. Server waken up and works now at %5.5f" % env.now)

    def done(self):
        self.servproc.interrupt()


class Terminal:
    def __init__(self, tid, env, server, nrjobs, lam, mu):
        self.id = tid
        self.server = server
        self.nrjobs = nrjobs
        self.interarrivaltime = 1/lam;
        self.servicetime = 1/mu;
        self.servproc = env.process(self.generatejobs(env))


        
    def generatejobs(self, env):
        i = 1
        while True:
            try:
                """ yield an event for new job arrival """
                job_interarrival = random.exponential( self.interarrivaltime )
                yield env.timeout( job_interarrival )  # event: job's arrival


                """ generate service time and add job to the list """
                job_duration = random.exponential( self.servicetime )

                self.server.Jobs.append(
                    Job(self.id, "Job %s" % i, env.now, job_duration)
                )
                # print( 'Terminal: %d job %d: arrive at = %5.5f, length = %5.5f' %( self.id, i, env.now, job_duration ) )
                i += 1

                """ if server is idle, wake it up """
                if not self.server.serversleeping.triggered:
                    self.server.serversleeping.interrupt("Wake up, please.")

                self.isWaiting = True
                # print("Terminal %d start waiting" % self.id)
                self.waitrtproc = env.process(self.waiting(env))
                yield self.waitrtproc
                # print("Terminal %d stop waiting" % self.id)
            except:
                return
            
    def waiting(self, env):
        try:
            yield (env.timeout(MAXSIMTIME))
        except simpy.Interrupt:
            pass
    
    def done(self):
        self.servproc.interrupt()
        
def survey(njob, l, m):
    POPULATION = njob
    LAMBDA = l
    MU = m
    
    env = simpy.Environment()
    MyServer = Server( env, 'FIFO' )

    MyJobGenerator = Terminal( "1",env, MyServer, POPULATION, LAMBDA, MU )

    env.run()

survey(100, 5, 8)


No. Terminal:  1
No. Jobs    :  1000
Server is idle at 0.00000
A new job comes. Server waken up and works now at 0.06566
Server is idle at 0.59441
A new job comes. Server waken up and works now at 0.65788
Server is idle at 2.61781
A new job comes. Server waken up and works now at 2.68218
Server is idle at 3.32229
A new job comes. Server waken up and works now at 3.38664
Server is idle at 5.04810
A new job comes. Server waken up and works now at 5.11132
Server is idle at 5.91453
A new job comes. Server waken up and works now at 5.98083
Server is idle at 7.50524
A new job comes. Server waken up and works now at 7.56803
Server is idle at 9.33087
A new job comes. Server waken up and works now at 9.39362
Server is idle at 9.80634
A new job comes. Server waken up and works now at 9.87227
Server is idle at 10.20154
A new job comes. Server waken up and works now at 10.26431
Server is idle at 10.93012
A new job comes. Server waken up and works now at 10.99329
Server is idle at 11.39026
A new jo

Server is idle at 603.64832
A new job comes. Server waken up and works now at 603.71419
Server is idle at 605.13138
A new job comes. Server waken up and works now at 605.19436
Server is idle at 606.37770
A new job comes. Server waken up and works now at 606.44337
Server is idle at 607.44362
A new job comes. Server waken up and works now at 607.50699
Server is idle at 608.25790
A new job comes. Server waken up and works now at 608.32378
Server is idle at 608.73613
A new job comes. Server waken up and works now at 608.80001
Server is idle at 609.22307
A new job comes. Server waken up and works now at 609.28761
Server is idle at 609.62047
A new job comes. Server waken up and works now at 609.68508
Server is idle at 610.21132
A new job comes. Server waken up and works now at 610.27458
Server is idle at 612.20919
A new job comes. Server waken up and works now at 612.27416
Server is idle at 613.92391
A new job comes. Server waken up and works now at 613.98828
Server is idle at 614.39694
A ne

Server is idle at 939.96594
A new job comes. Server waken up and works now at 940.03055
Server is idle at 940.40688
A new job comes. Server waken up and works now at 940.47301
Server is idle at 941.01651
A new job comes. Server waken up and works now at 941.08147
Server is idle at 941.69680
A new job comes. Server waken up and works now at 941.76325
Server is idle at 942.57622
A new job comes. Server waken up and works now at 942.63864
Server is idle at 943.05796
A new job comes. Server waken up and works now at 943.12405
End at 944.03104805


0.0